## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["contract_creation"],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-05 15:36:05 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=daily_address_summary template=transaction_fees
2024-12-05 15:36:05 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=daily_address_summary template=daily_address_summary
2024-12-05 15:36:05 [info     ] Rendering query                filename=querybuilder.py lineno=40 model=contract_creation template=contract_creation_traces
2024-12-05 15:36:05 [info     ] Loaded 2 python modules with intermediate model definitions. filename=registry.py lineno=62
2024-12-05 15:36:05 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-12-05 15:36:05 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76
2024-12-05 15:36:05 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18
2024-12-05 15:36:06 [info     ] Loaded OP chains metadata from /Users/pedro/github/ethereum-optimism/op-anal

## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="contract_creation",
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-05 15:36:08 [info     ] Executing model...             filename=testutils.py lineno=226
2024-12-05 15:36:08 [info     ] duckdb dataset='ingestion/traces_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/traces_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=63
2024-12-05 15:36:09 [info     ] registered view: 'ingestion_traces_v1' using 1 parquet paths filename=client.py lineno=53
2024-12-05 15:36:09 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=63
2024-12-05 15:36:11 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53
dict_keys(['create_traces_v1'])


┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ contract_creation_traces  │
│ create_traces_v1          │
│ ingestion_traces_v1       │
│ ingestion_transactions_v1 │
└───────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM create_traces_v1 LIMIT 10")

┌─────────┬──────────┬─────────┬────────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────┬────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Check the data output size

In [4]:
duckdb_client.sql("SELECT COUNT(*) FROM create_traces_v1")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        33413 │
└──────────────┘

In [6]:
duckdb_client.sql("SELECT input FROM ingestion_traces_v1 LIMIT 10")


┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                                                                                                                                                                                        

### You can also convert the results to dataframes to inspect them in more familiar ways

In [7]:
duckdb_client.sql("SELECT * FROM create_traces_v1 LIMIT 10").pl().head()

network,chain_id,chain,dt,block_timestamp,block_number,block_hash,transaction_hash,transaction_index,tr_from_address,tx_from_address,contract_address,tx_to_address,value_64,value_lossless,code,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,tx_method_id,code_bytelength
str,i32,str,date,u32,i64,str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,str,f64
"""mainnet""",10,"""op""",2024-11-18,1731890627,128145925,"""0xf43fe8bcfb3bd50a6d2a4dcefe0a…","""0x0b14740dd518911a9be9b14bd775…",17,"""0x4e1dcf7ad4e460cfd30791ccc4f9…","""0xc97240c92596276b8b9366064123…","""0xf40496ad3f6b5ef77a7151577953…","""0x24f6f36a551fe6008fa80afcff1d…",0,"""0""","""0x6080604052348015610010576000…","""0x608060405273ffffffffffffffff…","""create2""","""""","""""",258177,56609,0,"""0,0""","""""",1,"""0x540817d7""",518.0
"""mainnet""",10,"""op""",2024-11-18,1731890627,128145925,"""0xf43fe8bcfb3bd50a6d2a4dcefe0a…","""0x0b14740dd518911a9be9b14bd775…",17,"""0x4e1dcf7ad4e460cfd30791ccc4f9…","""0xc97240c92596276b8b9366064123…","""0xb4cc7b7fcd1c79b21d0daaf520cd…","""0x24f6f36a551fe6008fa80afcff1d…",0,"""0""","""0x6080604052348015610010576000…","""0x608060405273ffffffffffffffff…","""create2""","""""","""""",260638,56609,0,"""1,0""","""""",1,"""0x540817d7""",518.0
"""mainnet""",10,"""op""",2024-11-18,1731890629,128145926,"""0x00f2140ae46b91ba9cba478def59…","""0xb7a0a8d602ef4283b875ea0246d2…",12,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0x52aac8613345af8e9426d22c7d53…","""0xb23d757bc355fe035945b1c0119f…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",19629380,9031,0,"""0""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""",2024-11-18,1731890629,128145926,"""0x00f2140ae46b91ba9cba478def59…","""0xb7a0a8d602ef4283b875ea0246d2…",12,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0x52aac8613345af8e9426d22c7d53…","""0x1c7c4f14d09a059fa7d4cac92af1…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",19449076,9031,0,"""2""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""",2024-11-18,1731890629,128145926,"""0x00f2140ae46b91ba9cba478def59…","""0xb7a0a8d602ef4283b875ea0246d2…",12,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0x52aac8613345af8e9426d22c7d53…","""0x7144337b778b511531bf7dddeda8…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",19283143,9031,0,"""4""","""""",1,"""0xb1ae2ed1""",55.0


### Get table schema

In [8]:
duckdb_client.sql("DESCRIBE create_traces_v1")


┌───────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name    │ column_type │  null   │   key   │ default │  extra  │
│      varchar      │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ network           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ chain             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ dt                │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ block_timestamp   │ UINTEGER    │ YES     │ NULL    │ NULL    │ NULL    │
│ block_number      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ block_hash        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_hash  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ tr_from_ad